[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harpreetsahota204/medgemma_1_5/blob/main/using_medgemma_zoo_model.ipynb)


# Using MedGemma as Remotely Sourced Zoo Model

In [ ]:
import fiftyone as fo

from fiftyone.utils.huggingface import load_from_hub

dataset = load_from_hub(
    "Voxel51/MedXpertQA",
    name="MedXpertQA",
    max_samples=10,
    overwrite=True
    )

In [ ]:
dataset

# Setup Zoo Model

In [ ]:
import fiftyone.zoo as foz

foz.register_zoo_model_source("https://github.com/harpreetsahota204/medgemma_1_5", overwrite=True)

foz.download_zoo_model(
    "https://github.com/harpreetsahota204/medgemma_1_5",
    model_name="google/medgemma-1.5-4b-it", 
)

model = foz.load_zoo_model(
    "google/medgemma-1.5-4b-it",
    # install_requirements=True #run this to install requirements if they're not already
    )

# Use MedGemma for Classification

You can use this model for a zero-shot classification task as follows, which will add a [FiftyOne Classificaton](https://docs.voxel51.com/api/fiftyone.core.labels.html#fiftyone.core.labels.Classification) to your dataset:

In [ ]:
body_system_labels = dataset.distinct("body_system.label")

model.operation = "classify"

model.prompt = "As a medical expert your task is to classify this image into exactly one of the following body systems: " + ", ".join(body_system_labels)

dataset.apply_model(model, label_field="pred_body_system")

You can customize the system prompt as follows to guide the model's response. Note that we are using an existing field on the sample by passing `prompt_field="question"` into the [`apply_model`](https://docs.voxel51.com/api/fiftyone.core.dataset.html) method of the [Dataset](https://docs.voxel51.com/api/fiftyone.core.dataset.html)

In [ ]:
model.operation="classify"

model.system_prompt = """You have expert-level medical knowledge in radiology, histopathology, ophthalmology, and dermatology.

You are presented with the medical history of a patient and an accompanying image related to the patient. 

You will be asked a multiple question and are required to select from one of the available answers.

Once you have the answer, you  must respond in the following format:

```json
{
    "classifications": [
        {
            "label": "your single letter answer to the question",
            ...,
        }
    ]
}
```

Always return your response as valid JSON wrapped in ```json blocks and respond only with a one letter answer (one of A, B, C, D, E).
"""


dataset.apply_model(
    model, 
    label_field="model_answer", 
    prompt_field="question"
    )

# Using MedGemma for VQA

You can use the model for visual question answering as shown below. This example will use the same prompt on each [Sample](https://docs.voxel51.com/api/fiftyone.core.sample.html#module-fiftyone.core.sample) in the Dataset:

In [ ]:
model.operation="vqa"

model.prompt = "Describe the condition the image you see here"

dataset.apply_model(model, label_field="response")

This is using the default system prompt, which you can inspect as follows:

In [ ]:
print(model.system_prompt)

You can customize the system prompt as shown above, by simply following this pattern:

```python
model.system_prompt = "An awesome customized system prompt"
```

You can use a [Field](https://docs.voxel51.com/api/fiftyone.core.fields.html#module-fiftyone.core.fields) from the sample as a prompt by passing in `prompt_field=<field-name>` as shown below:

In [ ]:
model.operation="vqa"

dataset.apply_model(model, label_field="verbose_ans", prompt_field="question")